# Exploratory Analysis & Testing of Model
Notebook for testing and de-bugging the model class from user_knn.py

In [1]:
import json
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import src.recommend.user_knn as uk
import tqdm
import warnings

## Overview
- [TODO](#todo)
- [Test with toy dataset](#test-with-toy-dataset)
- [Test with music sample](#test-with-music-sample)
- [Analyze full music sample](#analyze-full-music-sample) . 

## TODO 
- ~~fix break in preds for toy data 0,6 (example from )~~ DONE: accidentaly abs on R matrix in wrong place.  
- Why are some columns/rows filled with almost all nans? If only 1 review, it won't work
- create function for train/test split 
- link albums id's to album names

## Test with toy dataset

In [2]:
def convert_vector(b):
    n = len(b)
    valid_idx = np.where(~np.isnan(b))[0] #2d vector of nx1, only want row index of valid values
    print(f'valid_idx{valid_idx}')
    D0 = np.ones([n, len(valid_idx)])
    for i, val in enumerate(valid_idx):
        D0[val, i] = np.nan
    BD0 = b * D0 # broacast
    return BD0

In [ ]:
A = np.array([
    [1, 4, 2, 2],
    [5, 2, 4, 4],
    [0, 0, 3, 0],
    [2, 5, 0, 5],
    [4, 1, 0, 1],
    [0, 2, 5, 0]
])
A = A.astype('float')
A[A == 0] = np.nan
A

### Correlation testing

In [ ]:
b = np.array([[1],[4],[np.nan],[2],[np.nan],[np.nan]])

B = convert_vector(b)
#n_B, p_B = B.shape
#D0 = np.ones([n_B, n_B])
#np.fill_diagonal(D0, 0)
#B = B.astype('float')
#B[B == 0] = np.nan
#BD0 = B*D0
#BD0[BD0==0] = np.nan
#BD0

In [ ]:
B

In [ ]:
np.where(~np.isnan(b))[0]

#### Block Matrix method
This is really inefficient because only ~1/4 of the calculations are going to be used

In [ ]:
R_AB = np.array(pd.concat([pd.DataFrame(A),pd.DataFrame(B)], axis=1).corr())
pd.DataFrame(R_AB)


In [ ]:
R_mini = pd.DataFrame(R_AB[:A.shape[1], A.shape[1]:])

#### Looping Method

In [ ]:
R_AB = []
for i in range(B.shape[1]):
    R_AB.append(np.array(pd.DataFrame(A).corrwith(pd.DataFrame(B)[i])))
np.array(R_AB).T

#### Lambda Method

In [ ]:
pd.DataFrame(B).apply(lambda s: pd.DataFrame(A).corrwith(s))

Instead of block matrix style, we can loop through each and calculate

### Setup

In [ ]:
uk_mod = uk.UserKNN(A)
uk_mod.B = B
uk_mod.b_valid_idx = np.where(~np.isnan(b))[0]

#### Toy test - strong generalization

In [ ]:
uk_mod.gen_M(strong=True)
uk_mod.gen_mu(strong=True)
uk_mod.gen_corrcoef(strong=True)
uk_mod.R_strong

In [ ]:
uk_mod.gen_preds(strong=True)
uk_mod.P_strong

### Weak Generalization

In [ ]:
uk_mod.gen_valid_idx()

In [ ]:
uk_mod.A_valid_idx

In [ ]:
uk_mod.train_test_split()

In [ ]:
uk_mod.A_test_idx

In [ ]:
uk_mod.A_prime

In [ ]:
uk_mod.gen_M()
uk_mod.gen_mu()

In [ ]:
uk_mod.gen_corrcoef()
uk_mod.R.round(2)

In [ ]:
uk_mod.gen_preds()
uk_mod.P.round(3)

In [ ]:
uk_mod.A

In [ ]:
uk_mod.A_prime

In [ ]:
uk_mod.P

In [ ]:
for t in uk_mod.A_test_idx:
    i, j = t
    print(f'True value: {uk_mod.A[i,j]}')
    print(f'Pred value: {uk_mod.P[i,j]}')
    print(f'Delta: {abs(uk_mod.A[i,j] - uk_mod.P[i,j])}')

### Strong Generalization

In [ ]:
uk_mod.gen_mu(strong=True)
uk_mod.mu_strong

In [ ]:
uk_mod.gen_corrcoef(strong=True)
uk_mod.R_strong

In [ ]:
uk_mod.gen_preds(strong=True)
uk_mod.P_strong

In [ ]:
for i, val in enumerate(b.tolist()[0]):
    if val == 0:
        continue
    y_true = val 
    y_hat = uk_mod.P_strong[i]
    delt = y_true - y_hat
    print(f'true val {y_true}')
    print(f'pred val {y_hat}')
    print(f'delta {delt}')

In [ ]:
b

## Test with Music Sample

In [ ]:
raw_users = [json.loads(line) for line in open('users.jsonl', 'r')]

In [ ]:
random.seed(37)
sample_frac = 1/10
N = round(len(raw_users) * sample_frac)
idx = random.sample(range(len(raw_users)), N)
raw_users_sample = [raw_users[i] for i in idx]

In [ ]:
n = 600000
p = len(raw_users_sample)

A = np.empty((n, p))
A[:] = np.nan


In [ ]:
raw_users[0].keys()

In [ ]:
col_index = []
pop_cols =[]
row_set = set()
id_ceiling = np.inf
max_id = 0
for j, u in enumerate(raw_users_sample):
    reviews = u['reviews']
    #print(reviews)
    add_rating = False
    for r in reviews:
        id = r['album_id']
        if id < id_ceiling:
            add_rating = True
            row_set.add(id)
            if max_id < int(id):
                max_id = id
            A[id, j] = r['rating']
    if add_rating:
        pop_cols.append(j)
        col_index.append(u['user_id'])
print(max_id)


In [ ]:
len(row_set)

In [ ]:
len(pop_cols)

In [ ]:
p

In [ ]:
np.sum(~np.isnan(A)) / A.size

In [ ]:
np.sum(~np.isnan(A))

In [ ]:
C = A[[i for i in row_set]]

In [ ]:
C

In [ ]:
C = C[:,[j for j in pop_cols]] #

In [ ]:
C.shape

How many have 1 review?

In [ ]:
C_num_alb_rev = np.sum(~np.isnan(C), axis=1)

In [ ]:
np.sum(C_num_alb_rev == 1) / len(C_num_alb_rev)

In [ ]:
np.sum(C_num_alb_rev < 2) / len(C_num_alb_rev)

In [ ]:
np.sum(~np.isnan(C))

In [ ]:
music_mod = uk.UserKNN(C)

In [ ]:
music_mod.gen_corrcoef()

Test gen_corrcoef()

In [ ]:
music_mod.R

In [ ]:
x = np.ma.masked_invalid(music_mod.A[:, 1])
y = np.ma.masked_invalid(music_mod.A[:, 2])
np.ma.corrcoef(x, y, rowvar=False)

In [ ]:
C_df = pd.DataFrame(C)

In [ ]:
#test_R = pd.DataFrame(C).corr().to_numpy()
#pd.DataFrame(test_R)

In [ ]:
#np.fill_diagonal(test_R, 0)
#test_R

In [ ]:
music_mod.gen_preds()

In [ ]:
np.sum(~np.isnan(music_mod.P))

In [ ]:
pd.DataFrame(music_mod.P * ~music_mod.M)

It seems some users are getting lots of nan's. Maybe because they don't have lots of album overlap?

In [ ]:
music_mod.A[:, 286]

In [ ]:
np.sum(~np.isnan(music_mod.A[:, 286]))

In [ ]:
n_reviews_per_album = np.sum(~np.isnan(music_mod.A), axis=1)
plt.hist(n_reviews_per_album, bins=50)

what percent have 1 review?

In [ ]:
pd.Series(n_reviews_per_album)[0:50]

gen_preds not working - investigate numerator and denominator of J_Bar

In [ ]:
np.histogram(music_mod.mu)

In [ ]:
music_mod.M

In [ ]:
D0 = np.ones([music_mod.n, music_mod.n])
np.fill_diagonal(D0, 0)



In [ ]:
A_zeros = np.nan_to_num(music_mod.A, copy=True, nan=0)
J_bar = (D0 @ A_zeros) / (D0 @ music_mod.M)
pd.DataFrame(J_bar)

In [ ]:
D = music_mod.M @ np.nan_to_num(abs(music_mod.R), copy=True, nan=0)

test_P = music_mod.mu + (np.nan_to_num(music_mod.A - np.nan_to_num(J_bar, copy=True, nan=0), nan=0) @ np.nan_to_num(music_mod.R, copy=True, nan=0)) / D
test_P

In [ ]:
A_zeros - np.nan_to_num(J_bar, copy=True, nan=0)

In [ ]:
print(f'R NAN {np.sum(np.isnan(music_mod.R))}')
print(f'A_zeros NAN {np.sum(np.isnan(A_zeros))}')
print(f'J_bar NAN {np.sum(np.isnan(J_bar))}')
print(f'R_zeros NAN {np.sum(np.isnan(np.nan_to_num(music_mod.R, copy=True, nan=0)))}')
print(f'test_P NAN {np.sum(np.isnan(test_P))}')

In [ ]:
pd.DataFrame(test_P)

In [ ]:
df = pd.DataFrame(music_mod.P)

## Analyze full music sample
- [Strong](#strong-generalization)

In [3]:
def create_A(user_reviews, min_revs=0, n=1000000):
    '''
    Iterate through user review json and create rating matrix A.
    '''
    p = len(user_reviews)
    A = np.empty((n, p))
    A[:] = np.nan
    col_index = [] # users in same order as columns
    row_set = set()
    for j, u in enumerate(user_reviews):
        reviews = u['reviews']
        if len(reviews) < min_revs:
            continue
        #print(reviews)
        col_index.append(u['user_id'])
        for r in reviews:
            id = r['album_id']    
            # TODO - check if id bigger than n
            row_set.add(id)
            A[id, j] = r['rating']  
    # exclude rows if empty
    row_index= sorted(row_set)
    A = A[row_index]
    return (A, row_index, col_index)


In [4]:
def sample_users(raw_users,s=37, sample_frac=0.01):
    '''
    Subset user json to random sample of size sample_frac * p, where p is number of user
    '''
    random.seed(s)
    N = round(len(raw_users) * sample_frac)
    idx = random.sample(range(len(raw_users)), N)
    raw_users_sample = [raw_users[i] for i in idx]
    return raw_users_sample

In [5]:
raw_users = [json.loads(line) for line in open('users.jsonl', 'r')]

In [6]:
user_sample = sample_users(raw_users, sample_frac=.01)

In [7]:
A, row_idx, col_idx = create_A(user_sample)
# 50sec to load 25% of users 
# local kernel crash with 50% of users  

In [8]:
# FULL MATRIX
#F, F_row_idx, F_col_idx = create_A(raw_users)

### Descriptive stats of sample
Percent non-sparse

In [ ]:
np.sum(~np.isnan(A)) / A.size

In [ ]:
n_rev_album = np.sum(~np.isnan(A), axis=1)
plt.hist(n_rev_album)

percent albums fewer reviews than n - AFFECTED BY SAMPLE FRACTION SIZE

In [ ]:
for i in range(10):
    print(f'% albums with {i} or fewer reviews')
    print(np.sum(n_rev_album <= i)/len(n_rev_album))


In [ ]:
n_rev_user = np.sum(~np.isnan(A), axis=0)
plt.hist(n_rev_user)

In [ ]:
for i in range(10):
    print(f'% users with {i} or fewer reviews')
    print(np.sum(n_rev_user <= i)/len(n_rev_user))

In [ ]:
A.shape

In [ ]:
len(col_idx)

In [ ]:
len(row_idx)

### Model sample
#### Weak Generalization

### 

In [ ]:
sample_mod = uk.UserKNN(A)

In [ ]:
sample_mod.gen_valid_idx()

In [ ]:
sample_mod.train_test_split()

In [ ]:
sample_mod.gen_M()
sample_mod.gen_mu()

NOTE: column sum of $M_i$ can be zero if random test ommitting removed only values in $A_i$ so that $A^{\prime}$ is all nan

In [ ]:
pd.DataFrame(sample_mod.mu)

In [ ]:
np.sum(np.sum(sample_mod.M, axis=0) ==0)

In [ ]:
sample_mod.gen_corrcoef()

In [ ]:
sample_mod.R

In [ ]:
sample_mod.gen_preds()

In [ ]:
i, j = sample_mod.A_test_idx[0]
sample_mod.P[i, j]

In [ ]:
len(sample_mod.test_idx)

In [ ]:
y_true = []
y_pred = []
error_list = []
for t in sample_mod.A_test_idx:
    i, j = t
    y = sample_mod.A[i,j]
    y_hat = sample_mod.P[i,j]
    y_true.append(y)
    y_pred.append(y_hat)
    delt = y - y_hat
    print(f'True value: {y}')
    print(f'Pred value: {y_hat}')
    print(f'Delta: {delt}')
    
    error_list.append(delt)


In [ ]:
print(f'average mean error: {np.nanmean(abs(np.array(error_list)))}')

A quarter of the preds are NaN
TODO - Look at a sample to see why it's not working 

In [ ]:
np.sum(np.isnan(error_list)) / len(error_list)

In [ ]:
plt.hist(error_list, bins=25)

#### Strong Generalization


In [9]:
b = A[:,-1]
np.sum(~np.isnan(b))
b = b.reshape([-1,1])
B = convert_vector(b)
A_tilde = np.delete(A, -1, axis=1)


valid_idx[   0   68  110  143  326  342  598  637  652  655  693  694  718  778
  791  884  901  914  937  940  991  992 1056 1077 1147 1152 1165 1198
 1205 1219 1228 1235 1236 1238 1278 1424 1461 1544 1572 1574 1583 1669
 1781 1825 1866 1882 1908 2005 2112 2120 2192 2494 2571 2771 2879 3104
 3370 3656 3718 3838 3930 3979 3980 4073 4220 4581 4600 4775 4974 5102
 5210 5218 5259 5262 5375 5401 5594 5668 5864 6589]


In [10]:
A.shape

(21809, 380)

In [11]:
#A_tilde10 = A_tilde[:,:10]
#A_tilde100 = A_tilde[:,:100]

In [12]:
uks = uk.UserKNN(A_tilde)
uks.B = B
uks.b_valid_idx = np.where(~np.isnan(b))[0]

In [14]:
len(uks.b_valid_idx)

80

In [15]:
uks.gen_M(strong=True)
uks.gen_mu(strong=True)

In [16]:
uks.mu_strong.shape

(80,)

In [17]:
uks.M_strong.shape

(21809, 379)

NOTE: gen_corrcoef had not terminated after 30 min on (nxp)=21000x300, test below 

In [18]:
uks.gen_corrcoef(strong=True)

/opt/homebrew/anaconda3/envs/sklearn-env/lib/python3.10/site-packages/numpy/lib/function_base.py:2845: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar, dtype=dtype)
/opt/homebrew/anaconda3/envs/sklearn-env/lib/python3.10/site-packages/numpy/lib/function_base.py:2704: RuntimeWarning: divide by zero encountered in divide
  c *= np.true_divide(1, fact)
/opt/homebrew/anaconda3/envs/sklearn-env/lib/python3.10/site-packages/numpy/lib/function_base.py:2845: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar, dtype=dtype)
/opt/homebrew/anaconda3/envs/sklearn-env/lib/python3.10/site-packages/numpy/lib/function_base.py:2704: RuntimeWarning: divide by zero encountered in divide
  c *= np.true_divide(1, fact)
/opt/homebrew/anaconda3/envs/sklearn-env/lib/python3.10/site-packages/numpy/lib/function_base.py:2845: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar, dtype=dtype)
/opt/homebrew/anaconda3/envs/sklearn-env/lib/python3.10/

In [19]:
uks.R_strong.shape

(379, 80)

In [20]:
uks.gen_preds(strong=True)

/Users/zmeitus/git-hub/music-recommendation/src/recommend/user_knn.py:141: RuntimeWarning: invalid value encountered in divide
  J_bar = (D0 @ A_zeros) / (D0 @ M)
/Users/zmeitus/git-hub/music-recommendation/src/recommend/user_knn.py:143: RuntimeWarning: invalid value encountered in divide
  P = mu + (np.nan_to_num(A - J_bar, nan=0) @ R_zeros) / D


In [32]:
np.nanmean(abs(uks.P_strong - b[uks.b_valid_idx]))

8.886124155516205

In [59]:
np.nanmean(abs(uks.P_strong - b[uks.b_valid_idx]))

8.886124155516205

In [23]:
np.sum(~np.isnan(uks.P_strong))

74

In [24]:
len(uks.P_strong)

80

In [61]:
strong_gen_errors = []
other_b_list =[]
for i, val in enumerate(uks.b_valid_idx):
    other_b_list.append(b[val])
    strong_gen_errors.append(uks.P_strong[i] - b[val])
np.nanmean(abs(uks.P_strong - b[uks.b_valid_idx]))

Mean absolute error

In [62]:
np.nanmean([abs(i) for i in strong_gen_errors])

#A10 - 13.24
#A100 - 9.302

9.501610247629287

Percent nans is high

In [63]:
np.sum(np.isnan(strong_gen_errors)) / len(strong_gen_errors)
#A10 - 66.25%
#A100 - 27.5%

0.075

Test correlation matrix speed performance

In [ ]:
#np.seterr(all="ignore")
warnings.filterwarnings('ignore')
R_list = []
A100 = A[:,:100]
#dfA_idx = pd.DataFrame(A[overlap_idx[0], A_idx])
B.shape[1]
for i in tqdm.tqdm(range(B.shape[1])):
    dfBi = pd.DataFrame(B[:, i])
    R_ABi = pd.DataFrame(A100).corrwith(dfBi[0])
    R_list.append(R_ABi)
    #print(np.sum(~np.isnan(R_ABi)))

#A10 - 1:15
#A100 - 7:36

In [ ]:
R_test = pd.DataFrame(B).apply(lambda s: pd.DataFrame(A100).corrwith(s))
#A10 - 1:07
#A100 - 7:08

In [ ]:
np.array(R_list).shape

In [ ]:
dfA= pd.DataFrame(A[:, A_idx])
dfB = pd.DataFrame(B[:, 0])
pd.DataFrame(A).corrwith(dfB[0])

In [ ]:
pd.DataFrame(B).apply(lambda s: pd.DataFrame(A).corrwith(s))

In [ ]:
pd.DataFrame([100, 100, 80]).corrwith(pd.DataFrame([30, 60, 60]))

In [ ]:
AB = np.concatenate([A100,B], axis=1)

In [ ]:
pd.DataFrame(AB).corr()